<a href="https://colab.research.google.com/github/T-Sawao/diveintocode-ml3/blob/main/term2_sprint13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.このSprintについて

**Sprintの目的**  
- フレームワークのコードを読めるようにする
- フレームワークを習得し続けられるようになる
- 理論を知っている範囲をフレームワークで動かす

**どのように学ぶか**  
TensorFLowのサンプルコードを元に、これまで扱ってきたデータセットを学習していきます。

# 2.コードリーディング

TensorFLowによって2値分類を行うサンプルコードを載せました。今回はこれをベースにして進めます。


tf.kerasやtf.estimatorなどの高レベルAPIは使用していません。低レベルなところから見ていくことにします。

## 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。


（例）


- 重みを初期化する必要があった
- エポックのループが必要だった

それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。


**データセットの用意**  
以前から使用しているIrisデータセットを使用します。以下のサンプルコードではIris.csvが同じ階層にある想定です。


Iris Species  
https://www.kaggle.com/uciml/iris/data

目的変数はSpeciesですが、3種類ある中から以下の2種類のみを取り出して使用します。  
- Iris-versicolor
- ris-virginica


### 1.1.1（解答）
- 重み、バイアスの初期化、更新
- エポックのループ
- 活性化関数を通して出力する。
- forward,back propagationを繰り返す。

## 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。


それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [ ]:
# モデルの作成
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    # バイアスの宣言
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    # レイヤーの構成
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

In [ ]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/diveintocode-ml/Iris.csv")
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

### 2.1.1(解答)　　
- forward,backward propagationの記述が無く簡素化された。
- 初期化、更新、活性化関数は定義と実行が１文でまとまられた。
- エポックのループ処理は変更なし。

# 3.他のデータセットへの適用

これまで扱ってきた小さなデータセットがいくつかあります。上記サンプルコードを書き換え、これらに対して学習・推定を行うニューラルネットワークを作成してください。

- Iris（3種類全ての目的変数を使用）
- House Prices

どのデータセットもtrain, val, testの3種類に分けて使用してください。




## 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成  
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。


Iris Species  
https://www.kaggle.com/uciml/iris/data

2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。


**《ヒント》**  
以下の2箇所は2クラス分類特有の処理です。

In [ ]:
# loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

# correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))

メソッドは以下のように公式ドキュメントを確認してください。


tf.nn.sigmoid_cross_entropy_with_logits  |  TensorFlow  
https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits

tf.math.sign  |  TensorFlow  
https://www.tensorflow.org/api_docs/python/tf/math/sign


＊tf.signとtf.math.signは同じ働きをします。

### 3.1.1（前処理）

In [ ]:
# データセットの読み込み
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/diveintocode-ml/Iris.csv")
# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = pd.get_dummies(y)
y = np.array(y)
X = np.array(X)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

### 3.2.1（解答）

In [ ]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3
# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 1.プレースホルダーの作成 (tensorflow2ではプレースホルダーは使用できない)
# 2.float型のNone×784の行列を作成

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 3.重みとバイアスを定義

# ネットワーク構造の読み込み                               
logits = example_net(X)
# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 4.モデルの実装

# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 5.モデルの評価

# 推定結果
correct_pred = tf.equal(tf.argmax(Y,1),tf.argmax(tf.nn.softmax(logits),1))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 6.変数（variables）の初期化

init = tf.global_variables_initializer()
# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 7.モデルのトレーニング

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

## 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。


House Prices: Advanced Regression Techniques  
https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。


分類問題と回帰問題の違いを考慮してください。

In [ ]:
# データセットの読み込み
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/diveintocode-ml/house-prices-advanced-regression-techniques/train.csv")
# データフレームから条件抽出
X = df[["GrLivArea", "YearBuilt"]]
y = df[["SalePrice"]]
y = np.array(np.log(y))
X = np.array(np.log(X))

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X_mms, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [ ]:
# 1.プレースホルダーの作成 (tensorflow2ではプレースホルダーは使用できない)
# 2.float型のNone×784の行列を作成

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

In [ ]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 1.プレースホルダーの作成 (tensorflow2ではプレースホルダーは使用できない)
# 2.float型のNone×784の行列を作成

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 3.モデルの作成 (⇒def example_net(x):)                             
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.square(logits - Y))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 4.モデルの評価

# 推定結果
correct_pred = logits

# 指標値計算
MSE = tf.reduce_mean(tf.square(correct_pred - Y))
# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 5.変数（variables）の初期化

init = tf.global_variables_initializer()
# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 6.モデルのトレーニング

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
      # エポックのループ
      total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
      total_loss = 0
      total_mse = 0
      for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
        # ミニバッチのループ
        sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
        loss, mse = sess.run([loss_op, MSE], feed_dict={X: mini_batch_x, Y: mini_batch_y})
        total_loss += loss
        total_mse += mse
      total_loss /= n_samples
      total_mse /= n_samples
      val_loss, val_mse = sess.run([loss_op, MSE], feed_dict={X: X_val, Y: y_val})
      # if epoch %10==0:
      print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, mse : {:.3f}, val_mse : {:.3f}".format(epoch, loss, val_loss, mse, val_mse))
    test_mse = sess.run(MSE, feed_dict={X: X_test, Y: y_test})
    print("test_mse : {:.3f}".format(test_mse))


### 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。


3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。


スクラッチで実装したモデルの再現を目指してください。

In [ ]:
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

# float化と0or1処理
X_train_flt = X_train.astype(np.float)
X_test_flt = X_test.astype(np.float)
X_train_flt /= 255
X_test_flt /= 255

# X_train = X_train_flt[:, np.newaxis,:,:]
# X_test = X_test_flt[:, np.newaxis,:,:]

# one hot処理
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_onehot = enc.fit_transform(y_train[:, np.newaxis])
y_test_onehot = enc.fit_transform(y_test[:, np.newaxis])
print(X_train.shape)
print(y_train_onehot.shape)

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 200
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train_onehot, batch_size=batch_size)

# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 1.プレースホルダーの作成 (tensorflow2ではプレースホルダーは使用できない)
# 2.float型のNone×784の行列を作成

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 3.重みとバイアスを定義

# ネットワーク構造の読み込み                               
logits = example_net(X)
# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 4.モデルの実装

# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 5.モデルの評価

# 推定結果
correct_pred = tf.equal(tf.argmax(Y,1),tf.argmax(tf.nn.softmax(logits),1))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 6.変数（variables）の初期化

init = tf.global_variables_initializer()
# ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
# 7.モデルのトレーニング
with tf.Session() as sess:
    sess.run(init)
    # エポックのループ
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
                    # -------
                    # 少数の切り上げ 
        total_loss = 0
        total_acc = 0
        # バッチのループ
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_onehot})
    print("test_acc : {:.3f}".format(test_acc))